<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Post-processing-the-reactive-island-data" data-toc-modified-id="Post-processing-the-reactive-island-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Post-processing the reactive island data</a></span><ul class="toc-item"><li><span><a href="#Loading-the-manifold-intersection-data-(warning:-takes-a-bit-of-time)" data-toc-modified-id="Loading-the-manifold-intersection-data-(warning:-takes-a-bit-of-time)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Loading the manifold intersection data (warning: takes a bit of time)</a></span></li><li><span><a href="#Plotting-reaction-fraction-for-small-excess-energy" data-toc-modified-id="Plotting-reaction-fraction-for-small-excess-energy-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Plotting reaction fraction for small excess energy</a></span></li><li><span><a href="#Loading-the-manifold-intersection-data-(warning:-takes-a-bit-of-time)" data-toc-modified-id="Loading-the-manifold-intersection-data-(warning:-takes-a-bit-of-time)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Loading the manifold intersection data (warning: takes a bit of time)</a></span></li><li><span><a href="#Plotting-reaction-fraction-for-large-excess-energy" data-toc-modified-id="Plotting-reaction-fraction-for-large-excess-energy-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Plotting reaction fraction for large excess energy</a></span></li></ul></li></ul></div>

# Post-processing the reactive island data

Different parameter combination of $(\zeta, \lambda)$ 

In [25]:
#!/usr/bin/env python3
"""
Created on Thu 18 July 2019 11:19:25

@author: Shibabrat Naik
"""

import numpy as np

from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pylab import rcParams
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
# mpl.rcParams['font.family'] = 'serif'
# mpl.rcParams['font.serif'] = ['Helvetica']

# plt.style.use('seaborn-dark')
plt.style.use('seaborn') # use sans-serif fonts

# plt.style.use('default')

rcParams['figure.figsize'] = 5, 5

label_size = 25 #10, 20
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size
mpl.rcParams['axes.labelsize'] = 25 #, 15

mpl.rcParams['axes.spines.left'] = True   ## display axis spines
mpl.rcParams['axes.spines.bottom'] = True
mpl.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['ytick.major.width'] = 1.0


import DeLeonBerne2dof
import importlib
importlib.reload(DeLeonBerne2dof)
import DeLeonBerne2dof as DB2dof



## Loading the manifold intersection data (warning: takes a bit of time)

In [27]:

# data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/data/DeLeon-Berne/' + \
#     'reaction-fraction/stable-mani-top-well/'
data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/system-bath/' + \
    'data-figures/DeLeon-Berne/reaction-fraction/stable-mani-top-well/'


# energy range discretization, this should match the data 
# deltaE_step = 0.01
# deltaE_start = 0.01;
# deltaE_stop = 0.51;
deltaE_step = 0.01
deltaE_start = 0.01;
deltaE_stop = 1.00;

deltaE_vals = np.arange(deltaE_start,deltaE_stop + deltaE_step,deltaE_step)

# zeta_lambda_comb = ['ZETA0.00-LAMBDA1.00/','fig3A1/','fig3A2/','fig3B1/','fig3B2/','fig3C1/','fig3C2/']
zeta_lambda_comb = ['ZETA0.00-LAMBDA1.00/','fig3A1/','fig3B2/','fig3C2/']

tubemani_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))
energysurf_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))

printFlag = False
for k in range(len(zeta_lambda_comb)):
    for i in range(len(deltaE_vals)):
    
        if (float(deltaE_vals[i]) - int(deltaE_vals[i]) < 1e-12):    
            data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                              'xeU1_stable_branch1_eqPt1_DelE%.0f'%(deltaE_vals[i]) + \
                              '_deleonberne.txt')
        else:
            try:
                data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                                  'xeU1_stable_branch1_eqPt1_DelE%.2f'%(deltaE_vals[i]) + \
                                  '_deleonberne.txt')
            except:
                data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                                  'xeU1_stable_branch1_eqPt1_DelE%.1f'%(deltaE_vals[i]) + \
                                  '_deleonberne.txt')
    

        tubemani_intersect_area[i,k] = DB2dof.get_area_closed_curve(data[:,(0,2)]) 

        # set parameters to calculate energy boundary on the section y = y_w
        params = set_parameters(zeta_lambda_comb[k])

        energy_boundary = DB2dof.get_energybndry_intersect_sos(params, 
                                                        deltaE_vals[i], printFlag)
        
        energysurf_intersect_area[i,k] = DB2dof.get_area_closed_curve(energy_boundary) 
        

## Plotting reaction fraction for small excess energy

In [30]:

%matplotlib
# print(tube_intersect_area)

ls_tick = 20
ls_axes = 25
mpl.rcParams['xtick.labelsize'] = ls_tick
mpl.rcParams['ytick.labelsize'] = ls_tick
mpl.rcParams['axes.labelsize'] = ls_axes


# markertype = ['^-c','s-r','v-g','h-b','d-k','<-m','>-']
if len(zeta_lambda_comb) == 4:
    markertype = ['^-c','s-r','d-k','>-']


fig = plt.figure(figsize = (10,10))
ax = fig.gca()

for k in range(len(zeta_lambda_comb)):
#     k = 3
    params = set_parameters(zeta_lambda_comb[k])
    
#     ax.plot(deltaE_vals, tubemani_intersect_area[:,k], '-')
    line_handle = ax.plot(params[2] + deltaE_vals, \
                    tubemani_intersect_area[:,k]/energysurf_intersect_area[:,k], \
                    markertype[k], \
                    label = r'$\zeta = %.2f,\lambda = %.2f$'%(params[4],params[5]))
    
    ax.legend(fontsize = ls_tick)


ax.set_ylim([-0.02, 0.52])
ax.set_xlabel(r'Total energy, $E$', fontsize = ls_axes)
# ax.set_ylabel(r'Reaction fraction, $T_{\rm react}(E)$', fontsize = ls_axes)
ax.set_ylabel(r'Microcanonical reaction probability, $T_{\rm react}(E)$', fontsize = ls_axes)

# fig.savefig('temp.pdf', bbox_inches = 'tight')
plt.show()



Using matplotlib backend: MacOSX


## Loading the manifold intersection data (warning: takes a bit of time)

In [ ]:


# data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/data/DeLeon-Berne/' + \
#     'reaction-fraction/stable-mani-top-well/'
data_path = '/Users/OptimusPrime/Documents/reaction-dynamics/system-bath/' + \
    'data-figures/DeLeon-Berne/reaction-fraction/stable-mani-top-well/'

# energy range discretization, this should match the data
deltaE_start = 1.1;
deltaE_stop = 6.5;
deltaE_step = 0.1;

deltaE_vals = np.arange(deltaE_start,deltaE_stop + deltaE_step,deltaE_step)

zeta_lambda_comb = ['ZETA0.00-LAMBDA1.00/','fig3A1/','fig3B2/','fig3C2/']

tubemani_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))
energysurf_intersect_area = np.zeros((len(deltaE_vals),len(zeta_lambda_comb)))

printFlag = False
for k in range(len(zeta_lambda_comb)):
    for i in range(len(deltaE_vals)):
    
        if (float(deltaE_vals[i]) - int(deltaE_vals[i]) < 1e-12):    
            data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                              'xeU1_stable_branch1_eqPt1_DelE%.0f'%(deltaE_vals[i]) + \
                              '_deleonberne.txt')
        else:
            data = np.loadtxt(data_path + zeta_lambda_comb[k] + \
                              'xeU1_stable_branch1_eqPt1_DelE%.1f'%(deltaE_vals[i]) + \
                              '_deleonberne.txt')
        
        

        tubemani_intersect_area[i,k] = DB2dof.get_area_closed_curve(data[:,(0,2)]) 

        # set parameters to calculate energy boundary on the section y = y_w
        params = set_parameters(zeta_lambda_comb[k])

        energy_boundary = get_energybndry_intersect_sos(params, 
                                                        deltaE_vals[i], printFlag)
        
        energysurf_intersect_area[i,k] = DB2dof.get_area_closed_curve(energy_boundary) 
        


## Plotting reaction fraction for large excess energy

In [ ]:

# print(tube_intersect_area)
%matplotlib

ls_tick = 20
ls_axes = 25
mpl.rcParams['xtick.labelsize'] = ls_tick
mpl.rcParams['ytick.labelsize'] = ls_tick
mpl.rcParams['axes.labelsize'] = ls_axes


markertype = ['^-c','s-r','d-k','>-']
fig = plt.figure(figsize = (10,10))
ax = fig.gca()

for k in range(len(zeta_lambda_comb)):
    params = DB2dof.set_parameters(zeta_lambda_comb[k])
    
#     ax.plot(deltaE_vals, tubemani_intersect_area[:,k], '-')
    line_handle = ax.plot(params[2] + deltaE_vals, \
                    tubemani_intersect_area[:,k]/energysurf_intersect_area[:,k], \
                    markertype[k], \
                    label = r'$\zeta = %.2f,\lambda = %.2f$'%(params[4],params[5]))
    
    ax.legend(fontsize = ls_tick)


ax.set_ylim([0.48,0.92])
ax.set_xlabel(r'Total energy, $E$', fontsize = ls_axes)
# ax.set_ylabel(r'Reaction fraction, $T_{\rm react}(E)$', fontsize = ls_axes)
ax.set_ylabel(r'Microcanonical reaction probability, $T_{\rm react}(E)$', fontsize = ls_axes)

# fig.savefig('temp.pdf', bbox_inches = 'tight')
plt.show()

